In [88]:
from lxml import etree, html
from lxml.html.clean import clean_html
import pandas as pd
from datetime import date, datetime
import requests
import re
import os
import shutil
from ASECORP_BBDD import tagea_BBDD_ASECORP, devuelve_patrones
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait       
from selenium.webdriver.common.by import By       
from selenium.webdriver.support import expected_conditions as EC
import time

# Mueve todos los fichero del directorio de trabajo a otro de archivo

# Recoge resumen diario del BOE de hoy

In [59]:

today = date.today()

# dd/mm/YYYY
hoy = today.strftime("%Y%m%d")
print("Fecha de Hoy =", hoy)

# dd
d = today.strftime("%d")
print("dia =", d)

# mm
m = today.strftime("%m")
print("mes =", m)

# YYYY
Y = today.strftime("%Y")
print("año =", Y)

print(today.strftime("%d/%m/%Y"))

Fecha de Hoy = 20210326
dia = 26
mes = 03
año = 2021
26/03/2021


In [60]:
def save_html(html, path):
    with open(path, 'wb') as f:
        f.write(html)

In [61]:
# Define expresiones REGEX para búsqueda de leyes, decretos, etc. referenciadas anteriormente
#pattern = ['Ley [0-9]+\/[0-9]+','Ley Orgánica [0-9]+\/[0-9]+','Decreto [0-9]+\/[0-9]+','Real Decreto [0-9]+\/[0-9]+','Real Decreto Legislativo [0-9]+\/[0-9]+','Real Decreto-ley [0-9]+\/[0-9]+','Orden [A-Z]+\/[0-9]+\/[0-9]+','Orden Circular [0-9]+\/[0-9]+','Reglamento \(UE\) [0-9]+\/[0-9]+', 'Reglamento de Ejeución \(UE\) [0-9]+\/[0-9]+' ,'Sentencia de [0-9]+ de [a-z]+ de [0-9]+','Sentencia [0-9]+\/[0-9]+','Orden de [0-9]+ de [a-z]+ de [0-9]+', 'Resolución de [0-9]+ de [a-z]+ de [0-9]+','Resolución [a-z]+\/[0-9]+\/[0-9]+', 'Nota de Servicio [0-9]+\/[0-9]+', 'Acuerdo multilateral M\-[0-9]+', 'Acuerdo Multilateral RID [0-9]+\/[0-9]+', 'Circular [0-9]+\/[0-9]+', 'Decisión \(UE\) [0-9]+\/[0-9]+', 'Decisión de Ejecución \(UE\) [0-9]+\/[0-9]+', 'Instrucción IS\-[0-9]+']
pattern = devuelve_patrones()

# ------------------------------------------------------------------------

# DOGC

In [62]:
URL_HTML_resumen = "https://dogc.gencat.cat/es/inici/"

In [63]:
### Necesita libreria Selenium para renderizar JS script

options = webdriver.ChromeOptions()
options.headless = True
driver = webdriver.Chrome(options=options)

driver.get(URL_HTML_resumen)
#print(driver.page_source)
response = driver.page_source

sumario_HTML = html.fromstring(response)
#print(sumario_HTML.xpath('//h2[@class="titol-component"]/a/@href'))
numero_diario = sumario_HTML.xpath('//h2[@class="titol-component"]/a/text()')
secciones = sumario_HTML.xpath('//div[@class="llistat_cont"]/ul/li/a/text()')
secciones_link = sumario_HTML.xpath('//div[@class="llistat_cont"]/ul/li/a/@href')
secciones_linkHTML = [ ('https://dogc.gencat.cat' + item) for item in secciones_link ]
# numero_pagina = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[3]/td[4]/text()')
# NID = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[2]/td[2]/text()')
# rango = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[5]/td[2]/text()')
# organo_emisor = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[7]/td[2]/text()')
# 
# print(numero_diario[0].strip(), numero_pagina[0].strip(), NID[0].strip(), rango[0].strip(), organo_emisor[0].strip())
# print()
print(numero_diario)
print(secciones)
print(secciones_linkHTML)

driver.quit()

['DOGC núm. 8374']
['Disposiciones', 'Cargos y personal', 'Anuncios de la Generalidad de Cataluña', 'Anuncios de la Administración local', 'Anuncios de la Administración de justicia ']
['https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8374#section-Disposiciones', 'https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8374#section-Cargos_y_personal', 'https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8374#section-Anuncios_de_la_Generalidad_de_Cataluña', 'https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8374#section-Anuncios_de_la_Administración_local', 'https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8374#section-Anuncios_de_la_Administración_de_justicia_']


In [64]:
#response

## Recoge Nombre Secciones Sumario

In [65]:
secciones = sumario_HTML.xpath('//div[@class="llistat_cont"]/ul/li/a/text()')
#print(secciones)
df_secciones_sumarios = pd.DataFrame()
for seccion in secciones:
    df_secciones_sumarios = df_secciones_sumarios.append({'Seccion': seccion}, ignore_index=True)


In [66]:
df_secciones_sumarios

,Seccion
0,Disposiciones
1,Cargos y personal
2,Anuncios de la Generalidad de Cataluña
3,Anuncios de la Administración local
4,Anuncios de la Administración de justicia


## Recoge Valores para formar URLs Secciones Sumario

In [67]:
URL_base_sumario = sumario_HTML.xpath('//div[@class="llistat_cont"]/ul/li/a/@href')
URL_sumario = [ ('https://dogc.gencat.cat' + item) for item in secciones_link ]

df_URL_sumarios = pd.DataFrame()

for URL_seccion in URL_base_sumario:    
    URL_sumario = 'https://dogc.gencat.cat' + str(URL_seccion)
    print(URL_sumario)
    df_URL_sumarios = df_URL_sumarios.append({'URL_Seccion': URL_sumario}, ignore_index=True)

https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8374#section-Disposiciones
https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8374#section-Cargos_y_personal
https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8374#section-Anuncios_de_la_Generalidad_de_Cataluña
https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8374#section-Anuncios_de_la_Administración_local
https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8374#section-Anuncios_de_la_Administración_de_justicia_


In [68]:
df_URL_sumarios

,URL_Seccion
0,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
1,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
2,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
3,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
4,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...


In [69]:
 df_sumarios = pd.concat([df_secciones_sumarios, df_URL_sumarios], axis=1)

In [70]:
df_sumarios

,Seccion,URL_Seccion
0,Disposiciones,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
1,Cargos y personal,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
2,Anuncios de la Generalidad de Cataluña,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
3,Anuncios de la Administración local,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
4,Anuncios de la Administración de justicia,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...


In [71]:
df_sumarios['URL_Seccion'][0]

'https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8374#section-Disposiciones'

## Recoge Items en Seccion Disposiciones

In [72]:
URL_HTML_disposiciones = df_sumarios[df_sumarios['Seccion'] == 'Disposiciones']

In [73]:
print(URL_HTML_disposiciones['Seccion'][0])

Disposiciones


In [74]:
print(URL_HTML_disposiciones['URL_Seccion'][0])

https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8374#section-Disposiciones


In [75]:
URL_HTML_disposiciones = URL_HTML_disposiciones['URL_Seccion'][0]

In [76]:
URL_HTML_disposiciones

'https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8374#section-Disposiciones'

In [77]:
options = webdriver.ChromeOptions()
options.headless = True
driver = webdriver.Chrome(options=options)

driver.get(URL_HTML_disposiciones)
#print(driver.page_source)
response = driver.page_source
### Recoge Items en Seccion Disposiciones
disposiciones_HTML = html.fromstring(response)

#print(html.tostring(disposiciones_HTML))
#print('____________________')

departamentos = disposiciones_HTML.xpath('/descendant::*[@class="wrapper-disposicions"][1]/h3/text()') # Sólo Sección disposiciones
disposiciones = disposiciones_HTML.xpath('/descendant::*[@class="wrapper-disposicions"][1]/ul/descendant::li/div/a/text()[1]')
disposiciones_link = disposiciones_HTML.xpath('/descendant::*[@class="wrapper-disposicions"][1]/ul/descendant::li/div/a/@href')
disposiciones_pdf = disposiciones_HTML.xpath('/descendant::*[@class="wrapper-disposicions"][1]/ul/descendant::li/div/div/a/@href')

# for i in range(len(departamentos)):
#     print(departamentos[i])
#     i += 1
#     #print(disposiciones_HTML.xpath('//*[@class="wrapper-disposicions"]/ul[' + str(i) + ']//li/div/a/text()'))
#     print(disposiciones_HTML.xpath('//*[@class="wrapper-disposicions"]/ul[1]//li/div/a/text()'))

print(departamentos)
print(disposiciones)
print(disposiciones_link)
print(disposiciones_pdf)

driver.quit()


['Departamento de la Presidencia', 'Departamento de Territorio y Sostenibilidad', 'Departamento de Agricultura, Ganadería, Pesca y Alimentación']
['CORRECCIÓN DE ERRATAS en el Decreto ley 5/2021, de 2 de febrero, por el que se aprueban medidas urgentes para la implementación y gestión de los fondos procedentes del Mecanismo de Recuperación y Resiliencia y del fondo REACT-EU para la Administración de la Generalidad de Cataluña y su sector público (DOGC núm. 8333, de 4.2.2021).', 'RESOLUCIÓN PRE/823/2021, de 23 de marzo, por la que se modifica la Resolución PRE/1846/2017, de 26 de julio, por la que se aprueban las bases reguladoras que tienen que regir el procedimiento de concesión de subvenciones para la organización y participación en actividades deportivas federadas.', 'RESOLUCIÓN TES/826/2021, de 22 de marzo, por la que se hace público el Acuerdo de informe de impacto ambiental del Proyecto de ampliación de una actividad de construcción, mantenimiento y reparación naval, en el términ

In [78]:
len(disposiciones)

7

In [79]:
#print(disposiciones[11])

In [80]:
for i in range(len(disposiciones)):
    print (i, disposiciones[i])

0 CORRECCIÓN DE ERRATAS en el Decreto ley 5/2021, de 2 de febrero, por el que se aprueban medidas urgentes para la implementación y gestión de los fondos procedentes del Mecanismo de Recuperación y Resiliencia y del fondo REACT-EU para la Administración de la Generalidad de Cataluña y su sector público (DOGC núm. 8333, de 4.2.2021).
1 RESOLUCIÓN PRE/823/2021, de 23 de marzo, por la que se modifica la Resolución PRE/1846/2017, de 26 de julio, por la que se aprueban las bases reguladoras que tienen que regir el procedimiento de concesión de subvenciones para la organización y participación en actividades deportivas federadas.
2 RESOLUCIÓN TES/826/2021, de 22 de marzo, por la que se hace público el Acuerdo de informe de impacto ambiental del Proyecto de ampliación de una actividad de construcción, mantenimiento y reparación naval, en el término municipal de Vilanova i la Geltrú (exp. ConsB19010).
3 RESOLUCIÓN TES/828/2021, de 22 de marzo, por la que se hace público el Acuerdo de declaraci

In [81]:
DOGC_sumarios = pd.DataFrame(columns=['item_Title','item_urlHTML','item_urlPDF'])

In [82]:
DOGC_sumarios

,item_Title,item_urlHTML,item_urlPDF


In [83]:
lista = []
for disposicion in disposiciones:
    lista.append(disposicion)

DOGC_sumarios['item_Title'] = lista

lista = []   
for link_HTML in disposiciones_link:
    #print(link)
    lista.append('https://dogc.gencat.cat' + str(link_HTML))

DOGC_sumarios['item_urlHTML'] = lista

lista = []   
for link_PDF in disposiciones_pdf:
    #print(link)
    lista.append(link_PDF)

DOGC_sumarios['item_urlPDF'] = lista

In [84]:
DOGC_sumarios

,item_Title,item_urlHTML,item_urlPDF
0,CORRECCIÓN DE ERRATAS en el Decreto ley 5/2021...,https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
1,"RESOLUCIÓN PRE/823/2021, de 23 de marzo, por l...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
2,"RESOLUCIÓN TES/826/2021, de 22 de marzo, por l...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
3,"RESOLUCIÓN TES/828/2021, de 22 de marzo, por l...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
4,"EDICTO de 16 de marzo de 2021, sobre un acuerd...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
5,"ORDEN ARP/68/2021, de 23 de marzo, por la que ...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
6,"ORDEN ARP/69/2021, de 23 de marzo, por la que ...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...


In [85]:
#DOGC_sumarios['item_urlHTML'][1]

In [93]:
### Recoge información de página de detalle con Selenium
### es necesario ya que la página se genera con un JS y
### hay que renderizarla con un headless web browser

options = webdriver.ChromeOptions()
options.headless = True
driver = webdriver.Chrome(options=options)

DOGC_sumarios['item_fullText'] = ''
DOGC_sumarios['item_urlXML'] = ''
DOGC_sumarios['tipo_doc'] = ''
DOGC_sumarios['fecha_doc'] = ''
DOGC_sumarios['numero_doc'] = ''
DOGC_sumarios['numero_control'] = ''
DOGC_sumarios['entidad_emisora'] = ''
DOGC_sumarios['CVE'] = ''
DOGC_sumarios['numero_DOGC'] = ''
DOGC_sumarios['fecha_DOGC'] = ''
DOGC_sumarios['seccion_DOGC'] = ''

for i, row in DOGC_sumarios.iterrows():
    # carga página HTML y genera árbol
    driver.get(row['item_urlHTML'])
    #print(driver.page_source)

    # clica sobre desplegable 'datos del documento' para generar contenido
    driver.find_element_by_xpath('//div[@class="panel-title"]/a[@href="#disposicions"]').click()
    time.sleep(5)

    #WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='panel-title']/a[@href='#disposicions']"))).click()

    response = driver.page_source
    sumario_HTML = html.fromstring(response)

    # Genera columnas de la tabla con Datos de los documentos
    item_urlXML = sumario_HTML.xpath('//*[@id="download"]/a[4]/@href')

    tipo_doc = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Tipo de documento")]/text()[2]')
    fecha_doc = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Fecha del documento")]/text()[2]')
    numero_doc = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Número de documento")]/text()[2]')
    numero_control = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Número de control")]/text()[2]')
    entidad_emisora = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Entidad emisora")]/text()[2]')
    CVE = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"CVE")]/text()[2]')
    numero_DOGC = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Número del DOGC")]/text()[2]')
    fecha_DOGC = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Fecha del DOGC")]/text()[2]')
    seccion_DOGC = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Sección del DOGC")]/text()[2]')

    DOGC_sumarios['item_urlXML'][i] = item_urlXML

    DOGC_sumarios['tipo_doc'][i] = tipo_doc
    DOGC_sumarios['fecha_doc'][i] = fecha_doc
    DOGC_sumarios['numero_doc'][i] = numero_doc
    DOGC_sumarios['numero_control'][i] = numero_control
    DOGC_sumarios['entidad_emisora'][i] = entidad_emisora
    DOGC_sumarios['CVE'][i] = CVE
    DOGC_sumarios['numero_DOGC'][i] = numero_DOGC
    DOGC_sumarios['fecha_DOGC'][i] = fecha_DOGC
    DOGC_sumarios['seccion_DOGC'][i] = seccion_DOGC

    # Recoge en una lista todos los parágrafos del texto completo
    fullText = []
    for div in sumario_HTML.xpath('//*[@id="fullText"]/div/div/div'):
        fullText.append(div.text_content())

    # Convierte la lista en una string
    fullText = ' '.join(fullText)

    DOGC_sumarios['item_fullText'][i] = fullText


driver.quit()

ElementClickInterceptedException: Message: element click intercepted: Element <a data-toggle="collapse" href="#disposicions" class="collapsed">...</a> is not clickable at point (392, 299). Other element would receive the click: <div class="loading" id="modalLoading">...</div>
  (Session info: headless chrome=89.0.4389.90)


In [ ]:
DOGC_sumarios

In [ ]:
DOGC_sumarios['item_fullText'][0]

In [ ]:
# Crea nueva columna vacía de tipo lista en tabla_analisis
#DOCM_sumarios['Referencias_completas'] = [[] for i in range(len(tabla_analisis))]
DOGC_sumarios['Tags'] = [[] for i in range(len(DOGC_sumarios))]
DOGC_sumarios['Match_ASECORP_BBDD'] = [[] for i in range(len(DOGC_sumarios))]

In [ ]:
pattern = ['Ley [0-9]+\/[0-9]+','Real Decreto [0-9]+\/[0-9]+','Real Decreto Legislativo [0-9]+\/[0-9]+','Real Decreto-ley [0-9]+\/[0-9]+','Orden [A-Z]+\/[0-9]+\/[0-9]+','Orden Circular [0-9]+\/[0-9]+','Reglamento \(UE\) [0-9]+\/[0-9]+', 'Reglamento de Ejeución \(UE\) [0-9]+\/[0-9]+' ,'Sentencia de [0-9]+ de [a-z]+ de [0-9]+','Sentencia [0-9]+\/[0-9]+','Orden de [0-9]+ de [a-z]+ de [0-9]+', 'Resolución de [0-9]+ de [a-z]+ de [0-9]+','Resolución [a-z]+\/[0-9]+\/[0-9]+', 'Nota de Servicio [0-9]+\/[0-9]+', 'Acuerdo multilateral M\-[0-9]+', 'Circular [0-9]+\/[0-9]+', 'Decisión \(UE\) [0-9]+\/[0-9]+', 'Decisión de Ejecución \(UE\) [0-9]+\/[0-9]+']

In [ ]:
for i, row in DOGC_sumarios.iterrows():
    # Busca expresiones REGX coincidentes con Patrones definidos
    DOGC_sumarios['Tags'][i] = re.findall('|'.join(pattern), str(row['item_fullText']), flags=re.IGNORECASE)
    print(DOGC_sumarios['Tags'][i])

In [ ]:
# Elimina Tags duplicados
for i, row in DOGC_sumarios.iterrows():
    DOGC_sumarios['Tags'][i] = list(set(DOGC_sumarios['Tags'][i]))
    print(DOGC_sumarios['Tags'][i])

In [ ]:
regex_result = []
[ regex_result.append(tag) for tags in DOGC_sumarios['Tags'] for tag in tags ]

In [ ]:
print(regex_result)

In [ ]:
## Elimina duplicados
boletin_flat_list = list(set(regex_result))

In [ ]:
boletin_flat_list

In [ ]:
# ## Importa BBDD ASECORP

# Inicializa datos de BBDD_ASECORP
boletin_ASECORP_flat_list = []
ASECORP_BBDD = pd.DataFrame()
ambitos = []

# Incluir en llamada a función el ambito territorial como lista
# si no se especifica ámbito los incluye todos
boletin_ASECORP_flat_list, ASECORP_BBDD, ambitos = tagea_BBDD_ASECORP(['España','Europa','Cataluña'])

print(ambitos)


In [ ]:
#boletin_ASECORP_flat_list

In [ ]:
## Busca coincidencias entre lista boletines BOEs explorados y lista boletines de BBDD ASECORP
set(boletin_flat_list) & set(boletin_ASECORP_flat_list)

In [ ]:
#DOCM_sumarios['Tags'].isin(ASECORP_BBDD_BOE['Tags'])
for row_to_compare in DOGC_sumarios['Tags']:
    for row_comparing in ASECORP_BBDD['Tags']:
        if set(row_comparing) & set(row_to_compare):
            print(set(row_comparing) & set(row_to_compare))

In [ ]:
for i, row_to_compare in DOGC_sumarios.iterrows():
    for j, row_comparing in ASECORP_BBDD.iterrows():
        if set(row_to_compare['Tags']) & set(row_comparing['Tags']):
            DOGC_sumarios['Match_ASECORP_BBDD'][i].append (ASECORP_BBDD['Codigo'][j])
            print(str(set(row_to_compare['Tags']) & set(row_comparing['Tags'])) + ' ' + str(row_comparing['Codigo']))

In [ ]:
DOGC_sumarios

# Genera Fichero EXCEL de resultados

In [ ]:
DOGC_sumarios_final = DOGC_sumarios

In [ ]:
DOGC_sumarios_final.dtypes

In [ ]:
## Cambia orden de columnas y elimina las no necesarias  
DOGC_sumarios_final.rename(columns={'CVE': 'Item_id', 'item_Title': 'Item_Title', 'item_urlPDF': 'PDF_Link', 'fecha_DOGC': 'Fecha_publicacion'}, inplace=True)
DOGC_sumarios_final = DOGC_sumarios[['Item_id','Item_Title','PDF_Link','Fecha_publicacion','Tags','Match_ASECORP_BBDD']]

In [ ]:
## Crea función que convierte lista a string en todas las columnas de DOCM_sumarios
## para evitar en presentación final los caracteres [' '] propios de las listas
def list2Str(lst):
    if type(lst) is list: # apply conversion to list columns
        return", ".join(lst)
    else:
        return lst

In [ ]:

DOGC_sumarios_final = DOGC_sumarios_final.apply(lambda x: [list2Str(i) for i in x])

In [ ]:
## Generar hyperlink a artículo BOE en CSV "=HYPERLINK("https://www.boe.es/boe/dias/2021/02/02/pdfs/BOE-A-2021-1474.pdf";"BOE-A-2021-1474")"
## https://www.boe.es/diario_boe/xml.php?id=## https://www.boe.es/boe/dias/2021/02/02/pdfs/BOE-A-2021-1474.pdf

DOGC_sumarios_final_CSV = DOGC_sumarios_final

In [ ]:
DOGC_sumarios_final_CSV['item_id'] = '=HIPERVINCULO(' + '"' + DOGC_sumarios_final_CSV['PDF_Link'] + '";' \
                                                        + '"' + DOGC_sumarios_final_CSV['Item_id'] + '")'      

In [ ]:
DOGC_sumarios_final_CSV['item_id'][0]

In [ ]:
# Elimina columna PDF_Link
DOGC_sumarios_final_CSV = DOGC_sumarios_final_CSV[['Item_id','Item_Title','Fecha_publicacion','Tags','Match_ASECORP_BBDD']]

In [ ]:
DOGC_sumarios_final_CSV

In [ ]:
DOGC_sumarios_final_CSV.to_csv("./ASECORP/Resultados_Matching_DOGC_" + today.strftime("%Y%m%d") + ".csv" ,index=False) 